In [4]:
!nvidia-smi

Fri Nov 22 19:14:03 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A40                     Off |   00000000:81:00.0 Off |                    0 |
|  0%   23C    P8             22W /  300W |       0MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import sys, getopt
import jsonlines
import random
import numpy as np
import pickle
from tqdm import tqdm

In [5]:
import pandas as pd
import re
import numpy as np
import os
import time
from openai import OpenAI
import pandas as pd
import jsonlines
import numpy as np
from tqdm import tqdm
import json
import tiktoken
import ollama
import csv
import copy
encoding = tiktoken.encoding_for_model("gpt-4")
import os
import sys
import logging
from tqdm import tqdm



model = 'llama3.1:70b'
num_ctx = 15000

In [6]:

###################
# Utility Functions
###################

def getWithoutAC(s):
    s = s[1:-1]
    s = s.split(", ")
    withoutAccess = []
    for k in s:
        if "[AC]" not in k:
            withoutAccess.append(k)
    return "<|>".join(withoutAccess)

def extractAC(s):
    s = s[1:-1]
    s = s.split(", ")
    ans = -1
    
    for k in s:
        if "[AC]" in k:
            match = re.search(r'\((\d+)\)', k)
            if match:
                number = int(match.group(1))
                ans = max(ans, number)
    m = {-1: 0, 0: 0, 1: 0, 2: 1, 3: 1}
    return m[ans]

def parse_data_string(dict_str):
    newDict = {}
    dict_str = dict_str[1:-1]
    dict_str = dict_str.split(',')
    for d in dict_str:
        newDict[d.split('=')[0].strip()] = d.split('=')[1].strip()
    return newDict

def getData(fileName):
# Read the file
    with open(fileName, "r") as file:
        lines = file.readlines()

    # Initialize variables
    current_ep = None
    all_entries = []

    # Iterate over the lines in the file
    for line in lines:
        # If the line starts with 'EP:', it's the start of a new EP block
        if line.startswith('EP:'):
            current_ep = line.strip().split('EP: ')[1]
        # If the line starts with 'ControlFlow:', it's the start of a new data block
        elif line.startswith('ControlFlow:'):
            current_block = {}
            current_block['EP'] = current_ep
            current_block['code'] = line.strip().split('ControlFlow: ')[1]
            all_entries.append(current_block)
        # elif line.startswith('Features:'):
        #     newDict = parse_data_string(line.strip().split('Features: ')[1])
        #     for k in newDict.keys():
        #         current_block[k] = newDict[k]
        #     all_entries.append(current_block)

    # Convert the list of blocks to a dataframe
    df = pd.DataFrame(all_entries)
    df["label"] = df["code"].apply(extractAC)
    df["code"] = df["code"].apply(getWithoutAC)
    return df

import numpy as np

def slidingWindow(path):
    window = []
    curPath = []
    for p in path:
        curPath.append(p)
        window.append(list(curPath))
    return window


def decompose(df):
    # Create a helper column for chunk grouping
    df['group'] = np.where(df['label'] == 0, np.arange(len(df)), df.groupby('EP').cumcount() // 4)

    # Group by 'EP', 'label' and 'group', and then merge 'subsequences'
    df_agg = df.groupby(['EP', 'label', 'group']).agg({
        'code': lambda x: '<PATH_SEP>'.join(x)
    })

    # Reset the index to get 'EP' and 'label' as columns
    df_agg.reset_index(inplace=True)

    # Drop the 'group' column
    df_agg.drop(columns=['group'], inplace=True)

    # If you still want to aggregate 'label' by taking the max value
    df_agg['label'] = df_agg.groupby('EP')['label'].transform('max')

    return df_agg



def shuffle_dataframes(df):
    # Concatenate the dataframes
    
    # Separate the labels from the features
    X = df.drop('label', axis=1)
    y = df['label']

    # Split the data into train and remaining data (test + validation)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Create train, test, and validation dataframes
    train_df = pd.concat([X_train, y_train], axis=1)
    test_df = pd.concat([X_test, y_test], axis=1)
    
    return train_df, test_df


# Creating the Training Data for the model
def createJsonL(df, fileName):
    fileName = fileName.replace(".txt", "")
    json_objects = []

    # Group the DataFrame by unique values in the 'EP' column
    grouped = df.groupby('EP')

    # Iterate over each group
    for _, group in grouped:
        # Shuffle the group randomly
        shuffled_group = group.sample(frac=1, random_state=42)  # Set a random_state for reproducibility

        # Keep at most 10 rows in the shuffled group
        # shuffled_group = shuffled_group.head(10)

        # Iterate over each row in the shuffled group
        for _, row in shuffled_group.iterrows():
            prompt = []

            # Iterate over each column (excluding 'label')
            for column in shuffled_group.columns:
                if column != 'label':
                    prompt.append(str(column) + ' = ' + str(row[column]))

            # Create the JSON object
            json_object = {
                'code': " ".join(prompt),
                'label': int(row['label'])
            }

            # Append the JSON object to the list
            json_objects.append(json_object)

    with jsonlines.open(fileName, 'w') as writer:
        writer.write_all(json_objects)


In [27]:
file_name = "data/aosp.txt"
try:
    df = getData(file_name)
except:
    print(f"Could not read {file_name}")

label_counts = df['label'].value_counts()

print("################\n1 - Protection Required\n0 - No Protection\n################\n")

print("LabelCounts:\n", label_counts)
merged_aaAdf = decompose(df)
merged_df = merged_aaAdf
trainDF, testDF = shuffle_dataframes(merged_df)

label_counts = merged_df['label'].value_counts()
print("LabelCounts:\n", label_counts)

print(merged_df.head())

createJsonL(trainDF, f'{file_name}_train.jsonl')
createJsonL(testDF, f'{file_name}_test.jsonl')

################
1 - Protection Required
0 - No Protection
################

LabelCounts:
 label
1    36207
0     9623
Name: count, dtype: int64
LabelCounts:
 label
1    14903
0     5242
Name: count, dtype: int64
                                 EP  label  \
0  abandonAudioFocus_AudioService_5      0   
1  abandonAudioFocus_AudioService_5      0   
2  abandonAudioFocus_AudioService_5      0   
3  abandonAudioFocus_AudioService_5      0   
4  abandonAudioFocus_AudioService_5      0   

                                                code  
0  [get]: MediaFocusControl var6 = this.mMediaFoc...  
1  [get]: MediaFocusControl var6 = this.mMediaFoc...  
2  [get]: MediaFocusControl var6 = this.mMediaFoc...  
3  [get]: MediaFocusControl var6 = this.mMediaFoc...  
4  [get]: MediaFocusControl var6 = this.mMediaFoc...  


In [6]:
# trainDF.iloc[0]['code']
def generate_java_method(name, code):
    # Extract the API name and method name dynamically
    api_name, method_name = name.split("_", 1)

    # Start the Java code formatting
    java_code = []
    java_code.append(f"// Method: {api_name}")
    java_code.append(f"// Service: {method_name}\n")
    java_code.append(f"public void {api_name}() {{")

    # Split and clean up the code
    lines = code.split("<|>")
    for line in lines:
        # Remove [..]: and clean up the lines
        cleaned_line = re.sub(r"\[.*?\]: ", "", line).strip()
        if cleaned_line:
            # Ensure only one semicolon is added
            if not cleaned_line.endswith(";"):
                cleaned_line += ";"
            java_code.append(f"    {cleaned_line}")

    java_code.append("}")
    return "\n".join(java_code)


# s= generate_java_method(trainDF.iloc[0]['EP'],trainDF.iloc[0]['code'])
# s
trainDF['java_code'] = trainDF.apply(lambda row: generate_java_method(row['EP'], row['code']), axis=1)
## DROP THE DUPLICATES USING LENGTH
trainDF['code_length'] = trainDF['java_code'].map(len)
trainDF = (
    trainDF.sort_values(by='code_length', ascending=False)  
    .drop_duplicates(subset=trainDF.columns[0])             
    .reset_index(drop=True)                                 
)
trainDF = trainDF.drop(columns=['code_length'])
trainDF['service_name'] = trainDF['EP'].apply(lambda word: word.split("_")[1])


In [ ]:
def run_first_prompt_Ollama(method_code,run):
    """ runs the first prompt - extract sinks from the traces
    """
    prompt1 = f"""
    You are an expert in the Android Framework and security, particularly focusing on access control for Android APIs. Your task is to determine if a given Android API needs to have an Access Control or not.
    I will provide you with the API name, parameters, and its static execution path. Your first task is to identify potential sink(s) in the API. You must place special attention to field access and updates as well as internal method invocations inside the API (e.g., native methods, file access methods, non-exposed service methods).
    Additionally, take hints from the method name, and if you encounter similar fields or method invocations in the traces, consider them as potential sinks, as they may relate to key functionality.
    
    Think like a human, break down each step in the execution to understand how data flows through the API, how internal fields are accessed or updated, and how different methods interact with sensitive data and provide reasoning.
    
    However, you should only return sinks that appear to handle sensitive information or may pose security/privacy risks. Avoid listing generic, non-sensitive operations, such as logging (e.g., accessing TAG fields), simple data type operations (e.g., Integer.valueOf, HashMap.get), string operations, basic object manipulations, locks, etc.

    For each potential sink you identify, provide detailed information in JSON format. Here are the categories and information fields:
    {{
    "Sinks": [
        {{
            "Type": "field",  
            "Trace_Event": "...",  # Field trace event indicating where the sink occurs, keep the event same as in the trace
            "AccessType": "get/set",  
        }},
        {{
            "Type": "method", 
            "Trace_Event": "...",  # Invoke trace event indicating where the sink occurs, keep the event same as in the trace
            "InvokedOn": "...",  
        }}
    ]
    }}

    InvokedOn should represent the field or object on which the method is invoked. If the method is invoked on an object that is retrieved dynamically (i.e., not through a field), or if there is no explicit field mentioned in the trace, leave InvokedOn as an empty string (""). Only fill "InvokedOn" when the trace **explicitly shows** the *name* of the field. Do not assume it; if it is unclear, leave it empty.

    Additionally, rank the sinks from the most sensitive to the least sensitive. The most sensitive sink should appear at the top of the list and the least sensitive at the bottom. Make sure the response includes a valid JSON and the reasoning for each sink. \n
    """
    
    
    prompt1 = f"""
            You are an expert in the Android Framework and security, particularly focusing on access control for Android APIs. Your task is to analyze the Java code of an Android API and determine whether it contains sinks that require access control.
            I will provide you with the API name and its code. Your task is to identify a set of potential sink(s) in the API. You must place special attention to field access and updates as well as internal method invocations inside the API (e.g., native methods, file access methods, non-exposed service methods).
            The instructions that form a sink may not always appear consecutively in the code. Analyze the flow of the API to identify the *complete set of instructions* that together contribute to a sink.
            
            Here are the positive rules that indicate potential sinks:
            Method invocations, especially native methods.
            Field get or set operations with sensitive naming or dataflow significance.
            Creation of new objects (new) with sensitive naming or dataflow links.
            Arguments and dataflow related to the API functionality.
            
            and negative rules:
            Generic utility methods (e.g., valueOf, toString, isEmpty).
            Logging methods, input validation/sanity checks.
            Auxiliary operations (e.g., locks, obtaining an instance of a class, or accessing context objects).
            
            Think like a human, break down each step in the execution to understand how data flows through the API, how internal fields are accessed or updated, and how different methods interact with sensitive data and provide reasoning.

            For each potential set of sinks you identify, provide information in JSON format. Here are the categories and information fields:
            {{
                "Sinks": [
                    [
                        code 1  # The first code that contributes to the sink
                        code 2  # Another relevant instruction (not necessarily consecutive)
                        code 3  # Continue listing all relevant code for this sink
                    ],
                    [
                        code 1  # Instructions for another sink
                        code 2
                        code 3
                    ]
                ]
            }}
            Each array inside "Sinks" represents a complete set of code instructions that collectively form a sink. Ensure each instruction is copied verbatim from the Java code provided.
            Additionally, rank the sinks from the most sensitive to the least sensitive. The most sensitive set of sink should appear at the top of the list and the least sensitive at the bottom. Make sure the response includes a valid JSON and the reasoning for each sink. \n
            """


    # user_prompt = get_method_traces_from_file(file_path, interface, method)
    
    user_prompt = method_code
    if(not run):
        return {
            "system_message": prompt1,
            "user_message": user_prompt,
            "response": "Not running"
        }
        
        
    
    response = ollama.chat(model=model, messages=[
    {
        'role': 'user',
        'content': prompt1 + user_prompt,
    },
    ]
    ,
     options={
        'num_ctx': num_ctx,
        'temperature': 0.3 
    }
    
    )
    
    # logging.info(f"Response for {method} = {response['message']['content']}")

    
    return {
        "system_message": prompt1,
        "user_message": user_prompt,
        "response": response['message']['content']
    }
    

# get the json from the res["response"]

def extract_json_from_string(input_string):
    """
    Extracts JSON from the given string.

    Args:
        input_string (str): The string containing embedded JSON.

    Returns:
        dict: The extracted JSON as a Python dictionary.
    """
    try:
        # Use a regex pattern to extract the JSON part
        json_pattern = r"```(?:json)?\n(.*?)\n```"
        match = re.search(json_pattern, input_string, re.DOTALL)
        if match:
            json_string = match.group(1)
            return json.loads(json_string)
        else:
            raise ValueError("No JSON found in the provided string.")
    except json.JSONDecodeError as e:
        raise ValueError(f"Error decoding JSON: {e}")

In [63]:


AMS_Df = trainDF[trainDF['service_name'] == 'ActivityManagerService'].copy()
AMS_Df.shape
AMS_Df['json_answer'] = None


output_folder = "./output"
counter = 0
for index, row in tqdm(AMS_Df.iterrows(), total=AMS_Df.shape[0], desc="Processing rows"):
    print(row['EP'])
    print(row['service_name'])
    res = run_first_prompt_Ollama(row['java_code'],True)
    json_answer = None
    try:
        json_answer = extract_json_from_string(res["response"])
    except Exception as e:
        print(f"Error extracting JSON from response: {e}")
        continue
        # Store the json_answer in the new column
    AMS_Df.at[index, 'json_answer'] = json_answer

    # Write response details to files
    folder_path = f'{output_folder}/{row["EP"].split("_")[0]}'
    os.makedirs(folder_path, exist_ok=True)

    with open(f'{folder_path}/system_message.txt', 'w') as f:
        f.write(res["system_message"])
    with open(f'{folder_path}/user_message.txt', 'w') as f:
        f.write(res["user_message"])
    with open(f'{folder_path}/response.txt', 'w') as f:
        f.write(res["response"])

pickle_file_path = "./AMS_Df.pkl"

# Serialize the DataFrame using pickle
with open(pickle_file_path, 'wb') as file:
    pickle.dump(AMS_Df, file)

print(f"DataFrame serialized and saved to {pickle_file_path}")

Processing rows:   0%|          | 0/192 [00:00<?, ?it/s]

bindService_ActivityManagerService_9
ActivityManagerService


Processing rows:   1%|          | 1/192 [01:30<4:48:06, 90.51s/it]

sendIdleJobTrigger_ActivityManagerService_1
ActivityManagerService


Processing rows:   1%|          | 2/192 [02:57<4:39:14, 88.18s/it]

...

checkPermissionWithToken_ActivityManagerService_5
ActivityManagerService


Processing rows:  99%|█████████▉| 191/192 [3:04:25<00:26, 26.66s/it]

checkPermission_ActivityManagerService_4
ActivityManagerService


Processing rows: 100%|██████████| 192/192 [3:04:54<00:00, 57.79s/it]

Error extracting JSON from response: Error decoding JSON: Expecting value: line 4 column 13 (char 39)
DataFrame serialized and saved to ./AMS_Df.pkl


In [9]:
# return {
#         "system_message": prompt1,
#         "user_message": user_prompt,
#         "response": response['message']['content']
#     }
# write res["response"] to a txt file
with open('output3.txt', 'w') as f:
    f.write(res["response"])
    f.close()

In [12]:
trainDF

word = "startRecentsActivity_ActivityTaskManagerService_4"
service = word.split("_")[1]
service 

'ActivityTaskManagerService'

In [16]:
trainDF.columns

Index(['EP', 'code', 'label', 'java_code', 'service_name'], dtype='object')

In [14]:
res["response"]

'After analyzing the provided Java code, I have identified several potential sinks that may require access control. Here are the results in JSON format:\n\n```\n{\n  "Sinks": [\n    [\n      "PackageManagerInternal var103 = var101.getPackageManagerInternalLocked();",\n      "ResolveInfo var105 = var103.resolveService(param2,param4,var100,var19,param7);"\n    ],\n    [\n      "IntentFirewall var421 = var420.mIntentFirewall;",\n      "boolean var425 = var421.checkService(var422,param2,param7,param6,param4,var423);"\n    ],\n    [\n      "AppOpsService var439 = var438.mAppOpsService;"\n    ]\n  ]\n}\n```\n\nRanking the sinks from most sensitive to least sensitive, I would place them in the following order:\n\n1. The first sink involves accessing the PackageManagerInternal instance and resolving a service using the resolveService method. This could potentially allow an attacker to access sensitive information about installed services or even install malicious services.\n2. The second sink 

In [ ]:




json_answer = extract_json_from_string(res["response"])

In [21]:
json_answer

{'Sinks': [['PackageManagerInternal var103 = var101.getPackageManagerInternalLocked();',
   'ResolveInfo var105 = var103.resolveService(param2,param4,var100,var19,param7);'],
  ['IntentFirewall var421 = var420.mIntentFirewall;',
   'boolean var425 = var421.checkService(var422,param2,param7,param6,param4,var423);'],
  ['AppOpsService var439 = var438.mAppOpsService;']]}

In [64]:
# read the serialized DataFrame
with open(pickle_file_path, 'rb') as file:
    AMS_Df = pickle.load(file)
    
AMS_Df.head()

,EP,code,label,java_code,service_name,json_answer
0,bindService_ActivityManagerService_9,"[inv]: int var12 = bindIsolatedService(param2,...",1,// Method: bindService\n// Service: ActivityMa...,ActivityManagerService,{'Sinks': [['ActivityManagerService var101 = t...
2,sendIdleJobTrigger_ActivityManagerService_1,[inv]: long var10 = Binder.clearCallingIdentit...,1,// Method: sendIdleJobTrigger\n// Service: Act...,ActivityManagerService,"{'Sinks': [['int var22 = broadcastIntent(0, va..."
7,startService_ActivityManagerService_7,[inv]: boolean var12 = hasFileDescriptors();<|...,1,// Method: startService\n// Service: ActivityM...,ActivityManagerService,{'Sinks': [['boolean var425 = var421.checkServ...
10,updateConfiguration_ActivityManagerService_2,[get]: ActivityTaskManagerService var3 = this....,1,// Method: updateConfiguration\n// Service: Ac...,ActivityManagerService,{'Sinks': [['SystemProperties.set(persist.sys....
22,getProcessMemoryInfo_ActivityManagerService_2,[inv]: long var6 = SystemClock.uptimeMillis();...,1,// Method: getProcessMemoryInfo\n// Service: A...,ActivityManagerService,"{'Sinks': [['Debug.getMemoryInfo(var60,var58);..."
